# The IVOA Registry

An International Virtual Observatory Alliance (IVOA) Registry is a database describing the resources provided by Virtual Observatory (VO) data providers from around the world.  

NAVO has an instance of an IVOA Registry known as the NAVO Directory.  This directory can be searched for resources that may be applicable to a particular scientific problem.

## Search the Registry Via a Web Page

This web page allows interactive searching for Virtual Observatory resources:
https://vao.stsci.edu/keyword-search/

## Search the Registry Programmatically

The registry can be searched via a Table Access Protocol (TAP) query.  TAP is a VO protocol that allows SQL-like queries to be executed on VO databases.

Below we define a convenience function for performing a registry TAP query, along with the necessary imports.

In [1]:
import requests
import io
from astropy.table import Table, unique

# For output using Markdown.
from IPython.display import Markdown, display

# There are a number of relatively unimportant warnings, so suppress them:
import warnings
warnings.filterwarnings("ignore")


In [2]:
def registry_tap_query(query):
    """
    Executes the specified query on the Registry TAP service.
    
    Parameters
    ----------
    query : string
        ADQL (SQL-like) to execute on the given TAP service.
        
    Returns
    -------
    astropy.table.Table
        An astropy table created from the contents of the VOTABLE response from the TAP query.
    """

    # Define the base URL for a TAP query on the registry.
    registry_tap_url = "https://vao.stsci.edu/RegTAP/TapService.aspx/sync"
    
    tap_params = {
        "request": "doQuery",
        "lang": "ADQL",
        "query": query
    }
    results = requests.post(registry_tap_url, data=tap_params)
    
    astropy_table = Table.read(io.BytesIO(results.content))
    return astropy_table

## Registry Schema
Of all the tables in the registry, the **resource**, **capability**, and **interface** tables (described below) are the most useful.  A resource can have one or more capabilities, which can each have one or more interfaces (although most capabilities only have one interface).

Full details of the registry schema can be found here: http://www.ivoa.net/documents/RegTAP/20141208/REC-RegTAP-1.0.pdf

### Resource Table
The **rr.resource** table describes registered data collections such as catalogs and surveys.  As an example, the registry TAP query below retrieves the first resource in the registry, the resource attributes are printed one per line. Perhaps the most useful attributes are:
* **ivoid** - A unique identifier (URI) for the resource.  Helps identify the source of the resource (e.g., "ivo://nasa.heasarc/skyview/goodsacisfb" identifies a resource at HEASARC).
* **content_type** - what sort of resource this is
* **waveband** - the waveband (or '#'-separated wavebands) of the data
* **short_name** - a short name of the resource (not always present)
* **res_title** - the title of the resource
* **res_description** - full description of the resource

In [3]:
query = """
    select top 1 
    ivoid, content_type, waveband, short_name, res_title, res_description
    from rr.resource
"""
results = registry_tap_query(query)

# Print column values, one per line.
for row in results:
    for i in range(0, len(row.colnames)):
        md = '**{}**: {}'.format(row.colnames[i], str(row[i], 'utf-8'))
        display(Markdown(md))
    print('=================================================================================')

**ivoid**: ivo://cxc.harvard.edu/csc

**content_type**: catalog

**waveband**: x-ray

**short_name**: CSC

**res_title**: Chandra Source Catalog

**res_description**: The Chandra X-ray Observatory is the U.S. follow-on to the Einstein Observatory and one of NASA"s Great Observatories. Chandra was formerly known as AXAF, the Advanced X-ray Astrophysics Facility, but renamed by NASA in December, 1998. Originally three instruments and a high-resolution mirror carried in one spacecraft, the project was reworked in 1992 and 1993. The Chandra spacecraft carries a high resolution mirror, two imaging detectors, and two sets of transmission gratings. Important Chandra features are: an order of magnitude improvement in spatial resolution, good sensitivity from 0.1 to 10 keV, and the capability for high spectral resolution observations over most of this range. The Chandra Source Catalog (CSC) includes information about X-ray sources detected in observations obtained using the Chandra X-ray Observatory. Release 1.0 of the catalog includes about 95,000 point and compact sources with observed spatial extents less than ~30 arcsec detected in a subset of ACIS imaging observations released publicly prior to the end of 2008. Observed source positions and multi-band count rates are reported, as well as numerous derived spatial, photometric, spectral, and temporal calibrated source properties that may be compared with data obtained by other telescopes. Each record includes the best estimates of the properties of a source based on data extracted from all observations in which the source was detected. The Chandra Source Catalog is extracted from the CXC"s Chandra Data Archive (CDA). The CXC should be acknowledged as the source of Chandra data. For detailed information on the Chandra Observatory and datasets see: http://cxc.harvard.edu/ for general Chandra information; http://cxc.harvard.edu/cda/ for the Chandra Data Archive; http://cxc.harvard.edu/csc/ for Chandra Source Catalog information.

### Capability Table
The **rr.capability** table describes the capabilities associated with resources.  For these purposes, we're interested in capabilities that describe web services that we can call programmatically, but capabilities can be other things such as human-readable web pages.

The most useful capability attributes are:

* **cap_type** - Desribes the type of capability.  For programmatic access, the useful values are:
 * **conesearch** - This is a standard Cone Search web service which allows searching a catalog of items by RA, Dec and radius (see [full documentation](http://www.ivoa.net/documents/latest/ConeSearch.html)).
 * **simpleimageaccess** - This is a standard query which allows searching a catalog of image products by RA, Dec, diameter, and image format (see [version 1 documentation](http://www.ivoa.net/documents/SIA/20091116/), and [version 2 documentation](http://www.ivoa.net/documents/SIA/20151223/)).
 * **simplespectralaccess** - This is a standard query which allows searching a catalog of spectral products by RA, Dec, diameter, and a handful of spectral parameters (see [full documentation](http://www.ivoa.net/documents/SSA/20120210/index.html))
 * **tableaccess** - This is a standard query which allows searching a remote database using SQL-like syntax (see [full documentation](http://www.ivoa.net/documents/TAP/20100327/))
<br><br>
* **standard_id** - Similar to *cap_type* but allows distinguishing between different versions of the same protocol.  These values are used with the above cap_type values:
 * **ivo://ivoa.net/std/conesearch** - for *cap_type* of *conesearch*
 * **ivo://ivoa.net/std/sia** - for *cap_type* of *simpleimageaccess*, protocol version 1
 * **ivo://ivoa.net/std/sia#query-2.0** - for *cap_type* of *simpleimageaccess*, protocol version 2
 * **ivo://ivoa.net/std/ssa** - for *cap_type* of *simplespectralaccess*
 * **ivo://ivoa.net/std/tap** - for *cap_type* of *tableaccess*
 
### Interface Table
In order to use a capability, one needs to know the URL of the service.  That information is available in the **access_url** field of the interface(s) associated with the capability.  

The example query below finds the capabilities of the Chandra resource found in the previous example, and also joins with the **rr.interface** table to get the **access_url** for those capabilities.

In [4]:
query = """
    select cap.cap_index, cap.cap_type, cap.standard_id, int.access_url from rr.resource res 
    natural join rr.capability cap
    natural join rr.interface int
    where res.ivoid = 'ivo://cxc.harvard.edu/csc'
"""
results = registry_tap_query(query)

results

cap_index,cap_type,standard_id,access_url
int16,object,object,object
1,conesearch,ivo://ivoa.net/std/conesearch,http://cda.harvard.edu/cscvo/coneSearch?
2,tableaccess,ivo://ivoa.net/std/tap,http://cda.harvard.edu/csctap
3,capability,ivo://ivoa.net/std/vosi#capabilities,http://cda.harvard.edu/csctap/capabilities
4,capability,ivo://ivoa.net/std/vosi#availability,http://cda.harvard.edu/csctap/availability
5,capability,ivo://ivoa.net/std/vosi#tables,http://cda.harvard.edu/csctap/tables


## More Sample Queries

#### Find all Simple Image Access services with 'Chandra' in the title.

In [5]:
query = """
    select res.res_title, res.ivoid, cap.cap_type, int.access_url from rr.resource res 
    natural join rr.capability cap
    natural join rr.interface int
    where 
    res.res_title like '%chandra%'
    and cap.cap_type = 'simpleimageaccess'
"""
results = registry_tap_query(query)

results.show_in_notebook()

idx,res_title,ivoid,cap_type,access_url
0,GOODS Chandra ACIS: Full band (0\.5\-8 keV),ivo://nasa.heasarc/skyview/goodsacisfb,simpleimageaccess,https://skyview.gsfc.nasa.gov/cgi-bin/vo/sia.pl?survey=goodsacisfb&amp;
1,Spitzer IRAC/MUSYC Public Legacy Survey in the Extended Chandra Deep Field South,ivo://irsa.ipac/spitzer/images/simple,simpleimageaccess,https://irsa.ipac.caltech.edu/cgi-bin/Atlas/nph-atlas?mission=SIMPLE&amp;hdr_location=%5CSIMPLEDataPath%5C&amp;SIAP_ACTIVE=1&amp;collection_desc=SIMPLE&amp;
2,GOODS Chandra ACIS: band ( keV),ivo://nasa.heasarc/skyview/goodsacisb,simpleimageaccess,https://skyview.gsfc.nasa.gov/cgi-bin/vo/sia.pl?survey=goodsacisb&amp;
3,LABOCA Extended Chandra Deep Field South Submillimetre Survey,ivo://nasa.heasarc/skyview/cdfs-less,simpleimageaccess,https://skyview.gsfc.nasa.gov/cgi-bin/vo/sia.pl?survey=cdfs-less&amp;
4,Chandra Observations,ivo://nasa.heasarc/chanmaster,simpleimageaccess,https://heasarc.gsfc.nasa.gov/xamin/vo/sia?table=chanmaster&amp;
5,Chandra Source Catalog,ivo://cxc.harvard.edu/csc.siap,simpleimageaccess,http://cda.harvard.edu/cscsiap/queryImages?
6,Chandra X-ray Observatory Data Archive,ivo://cxc.harvard.edu/cda.siap,simpleimageaccess,http://cda.harvard.edu/cxcsiap/queryImages?


#### Limit Chandra image service results to those from Harvard.

In [6]:
query = """
    select res.res_title, res.ivoid, cap.cap_type, int.access_url from rr.resource res 
    natural join rr.capability cap
    natural join rr.interface int
    where 
    res.res_title like '%chandra%'
    and cap.cap_type = 'simpleimageaccess'
    and res.ivoid like '%harvard%'
"""
results = registry_tap_query(query)

results.show_in_notebook()

idx,res_title,ivoid,cap_type,access_url
0,Chandra Source Catalog,ivo://cxc.harvard.edu/csc.siap,simpleimageaccess,http://cda.harvard.edu/cscsiap/queryImages?
1,Chandra X-ray Observatory Data Archive,ivo://cxc.harvard.edu/cda.siap,simpleimageaccess,http://cda.harvard.edu/cxcsiap/queryImages?


#### Find all values for 'waveband' that exist in the registry.

In [7]:
query = """
    select waveband, count(waveband) from rr.resource
    group by waveband
    order by waveband
"""
results = registry_tap_query(query)

results.show_in_notebook()

idx,waveband,COUNT
0,,0
1,euv,9
2,euv#uv,1
3,euv#x-ray,3
4,gamma-ray,201
5,gamma-ray#infrared,7
6,gamma-ray#infrared#radio,1
7,gamma-ray#optical,53
8,gamma-ray#optical#infrared,16
9,gamma-ray#optical#infrared#radio,3


#### Find image services for UV data served by STScI.

In [8]:
query = """
    select res.res_title, res.ivoid, cap.cap_type, int.access_url from rr.resource res 
    natural join rr.capability cap
    natural join rr.interface int
    where 
    cap.cap_type = 'simpleimageaccess'
    and res.waveband like '%uv%'
    and res.ivoid like '%stsci%'
"""
results = registry_tap_query(query)

results.show_in_notebook()

idx,res_title,ivoid,cap_type,access_url
0,ACS Galactic Globular Cluster Survey (ACSGGCT),ivo://mast.stsci/acsggct,simpleimageaccess,http://archive.stsci.edu/siap/search.php?id=acsggct&amp;
1,Hubble Infrared Pure Parallel Imaging Extragalactic Survey (HIPPIES),ivo://mast.stsci/hippies,simpleimageaccess,http://archive.stsci.edu/siap/search.php?id=HIPPIES&amp;
2,Ultraviolet Imaging Telescope (UIT),ivo://mast.stsci/siap/uit,simpleimageaccess,http://archive.stsci.edu/siap/search.php?id=UIT&amp;
3,Hubble Space Telescope Press Release Image Archive 2008-2010,ivo://mast.stsci/stpr,simpleimageaccess,http://archive.stsci.edu/stpr/vo_search.php?
4,GALEX Atlas of Nearby Galaxies,ivo://mast.stsci/siap/galex_atlas,simpleimageaccess,http://archive.stsci.edu/siap/search.php?id=galex_atlas&amp;
5,Cosmic Assembly Near-IR Deep Extragalactic Legacy Survey (CANDELS),ivo://mast.stsci/candels,simpleimageaccess,http://archive.stsci.edu/siap/search.php?id=candels&amp;
6,Panchromatic Hubble Andromeda Treasury (PHAT),ivo://mast.stsci/phat,simpleimageaccess,http://archive.stsci.edu/siap/search.php?id=phat&amp;
7,HST Ultraviolet Atlas of Nearby Galaxies,ivo://mast.stsci/siap/hst.maoz_atlas,simpleimageaccess,http://archive.stsci.edu/siap/search.php?id=maoz_atlas&amp;
8,Cluster Lensing And Supernova survey with Hubble (CLASH),ivo://mast.stsci/clash,simpleimageaccess,http://archive.stsci.edu/siap/search.php?id=clash&amp;
9,"Archive of Nearby Galaxies: Reduce, Reuse, Recycle (ANGRRR)",ivo://mast.stsci/angrrr,simpleimageaccess,http://archive.stsci.edu/siap/search.php?id=angrrr&amp;
